In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch

## Importing data and refining it

In [13]:
train_data = pd.read_csv('dataset/train.csv')
test_data = pd.read_csv('dataset/test.csv')

y_train  = train_data.pop('label')

X_train = train_data
X_test = test_data
train_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
nan_values = X_train.isna().any()
print("Columns with NaN values:")
print(nan_values[nan_values].index.tolist())

Columns with NaN values:
[]


In [15]:
data = torch.Tensor(np.array(train_data)).reshape(-1,28,28).unsqueeze(1).float()
target = torch.Tensor(np.array(y_train)).long()

In [16]:
data.shape,target.dtype

(torch.Size([42000, 1, 28, 28]), torch.int64)

In [17]:
# converting the data to tensor
X_train = torch.Tensor(np.array(X_train)).reshape(-1,28,28)
y_train = torch.Tensor(y_train)
X_train.shape

torch.Size([42000, 28, 28])

In [18]:
# Define a custom dataset class
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
class mnist_dataloader(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        target = self.targets[index]

        # Apply transformations if specified
        if self.transform:
            sample = self.transform(sample)

        return sample, target

transform = transforms.Compose([
    # transforms.ToTensor(),
    # transforms.Normalize((0.5,), (0.5,)),
    transforms.Lambda(lambda x: x.unsqueeze(0))
])

In [19]:
# using dataloader to process data further 
mnist_data = mnist_dataloader(X_train,y_train,transform=transform)
custom_dataloader = DataLoader(mnist_data, batch_size=4, shuffle=True)

In [20]:
# # Iterate over the data loader
# for batch_idx, (inputs, targets) in enumerate(custom_dataloader):
#     # Process the batch
#     print("Batch", batch_idx)
#     print("Inputs shape:", inputs.shape)
#     print("Targets shape:", targets.shape)

## Creating a model

In [21]:
import torch.nn as nn
class mnist_model(nn.Module):
    def __init__(self):
        super(mnist_model, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    
    def forward(self,x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = x.view(-1, 64 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.softmax(x)

        pass
        

In [22]:
co = nn.Conv2d(1,2,5)
X_train_reshaped = X_train.unsqueeze(1)#.unsqueeze(0)
pr = co(X_train_reshaped)
# X_train_reshaped[0].shape
pr.shape

torch.Size([42000, 2, 24, 24])

In [24]:
model = mnist_model()

In [ ]:
pre = model(X_train_reshaped[0])
pre

tensor([[ 3.5167, -2.8625,  3.5245,  3.9322, -6.0809,  4.4876,  2.2876,  3.7622,
          0.2538,  7.6842]], grad_fn=<AddmmBackward0>)

In [ ]:
for data, target in custom_dataloader:
    print(data.shape,target.shape)
    print(type(data),data.dtype)
    print(type(model(data)),model(data).dtype)
    print(type(target),target.dtype)
    break

torch.Size([4, 1, 28, 28]) torch.Size([4])
<class 'torch.Tensor'> torch.float32
<class 'torch.Tensor'> torch.float32
<class 'torch.Tensor'> torch.float32


/var/folders/cz/lky96vlx5b3gt97txms5jymm0000gn/T/ipykernel_22209/2615439669.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.softmax(x)


In [25]:
import torch.optim as optim
from tqdm import tqdm

In [26]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps')
device

device(type='mps')

In [30]:
# Define loss function and optimizer
batch_size = 64
learning_rate = 0.001
epochs = 2

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
i = 0
loss_list = []
for epoch in tqdm(range(epochs)):
  
  model.to(device)

  model.train()

  for data, target in custom_dataloader:
    data = data.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    output = model(data)
    target = target.to(torch.long)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

  loss_list.append(loss.item())
    # Print training progress (optional)
    # if (i+1) % 100 == 0:
    #   print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
    #       epoch + 1, epochs, i + 1, len(custom_dataloader.dataset), loss.item()))

# Test the model (optional)
# with torch.no_grad():
#   correct = 0
#   total = 0
#   for data, target in custom_dataloader:
#     output = model(data)
#     _, predicted = torch.max(output.data, 1)
#     total += target.size(0)
#     correct += (predicted == target).sum().item()

#   print('Accuracy of the network on the 10000 test images: {} %'.format(
#       100 * correct / total))

# Save the model (optional)
# torch.save(model.state_dict(), 'mnist_classifier.pt')

  0%|          | 0/2 [00:00<?, ?it/s]/var/folders/cz/lky96vlx5b3gt97txms5jymm0000gn/T/ipykernel_80919/2615439669.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.softmax(x)
100%|██████████| 2/2 [04:10<00:00, 125.49s/it]


In [32]:
loss_list

[2.4611501693725586, 2.2111501693725586]

In [37]:
def plot_loss(loss_list):
    l = len(loss_list)
    epo = [i for i in range(l)]
    plt.plot(epo,loss_list)

In [38]:
plot_loss(loss_list)

TypeError: 'float' object cannot be interpreted as an integer